# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatenating

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [1]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
outcomes = pd.read_csv(',
                       parse_dates=['DateTime', 'Date of Birth'])

In [2]:
outcomes = pd.read_csv('data/Austin_Animal_Center_Outcomes_041122.csv',
                       parse_dates=['DateTime', 'Date of Birth'])

FileNotFoundError: [Errno 2] No such file or directory: 'data/Austin_Animal_Center_Outcomes_041122.csv'

In [ ]:
outcomes.head()

In [ ]:
outcomes.info()

In [ ]:
# Let's create our Age in Days column
outcomes['Calculated Age in Days'] = outcomes['DateTime'].dt.normalize() - outcomes['Date of Birth']

In [ ]:
outcomes['Calculated Age in Days']

In [ ]:
# Grab just the integer here...
outcomes['Calculated Age in Days'] = outcomes['Calculated Age in Days'].dt.days

In [ ]:
# Sanity check
outcomes.head()

## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [ ]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby('Animal Type')

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is `groups`.

In [ ]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value

outcomes.groupby('Animal Type').groups

In [ ]:
# Same goes for multi-index groupbys
animal_outcome = outcomes.groupby(['Animal Type', 'Outcome Type'])

In [ ]:
# .groups outputs a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

In [ ]:
# We can then get a specific group, such as cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

## Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [ ]:
# Test it out
outcomes.groupby(by=['Animal Type', 'Outcome Type']).mean()

In [ ]:
outcomes['Animal Type'].value_counts()

## Exercise

Use `.groupby()` to find the most recent birth date of each (main) animal type.


In [ ]:
# Your code here
outcomes.groupby(by='Animal Type')['Date of Birth'].max()

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [ ]:
outcomes.groupby(by=['Outcome Type', 'Sex upon Outcome']).agg('mean')

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [ ]:
# Check it out!
outcomes.pivot_table(index='Outcome Type', columns='Sex upon Outcome', aggfunc='mean')

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [ ]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

In [ ]:
toy2

In [ ]:
# We can't just join these as they are, since we haven't specified our suffixes

toy1.join(toy2)

In [ ]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [ ]:
toy1.set_index('age')

In [ ]:
toy1.set_index('age').join(toy2.set_index('age'))

In [ ]:
toy1.drop('age', axis=1).join(toy2)

## `.merge()`

Or we could use `.merge()`:

In [ ]:
toy1.merge(toy2, on='age')

In [ ]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

In [ ]:
states = pd.read_csv('data/states.csv', index_col=0)
states

## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [ ]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

In [ ]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [ ]:
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

In [ ]:
ds_full = pd.concat([ds_chars, prefs])
ds_full

`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [ ]:
ds_full = pd.concat([ds_chars, prefs], axis=1)
ds_full

## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [ ]:
# Peek at the outcomes data we already had in here
outcomes.head()

In [ ]:
# Read in the intakes data
intakes = pd.read_csv("data/Austin_Animal_Center_Intakes_041122.csv")

# Check out the intakes data
intakes.head()

In [ ]:
# Let's try merging on Animal ID
merged = outcomes.merge(intakes, on= "Animal ID", how='inner', suffixes=('_outcome', '_intake'))

In [ ]:
# What was the result?
merged.head()

In [ ]:
# Check the shape...
merged.shape

In [ ]:
# And the shapes of the two original dataframes?
intakes.shape

In [ ]:
outcomes.shape

In [ ]:
merged.loc[merged['Animal ID'] == 'A720371']

Let's discuss/explore: did that work the way we expected?

- 


<details>
    <summary>Observation Notes</summary>

- We went from about 138k rows in each of the dataframes to 178k! Even using an inner join! Something seems off. 
    
    
</details>

In [ ]:
# We might want to try something different
# Can we clean something to make a better merge?
clean_outcomes = outcomes.drop_duplicates(subset=['Animal ID'], keep='last')

In [ ]:
clean_outcomes.head()

In [ ]:
clean_intakes = intakes.drop_duplicates(subset=['Animal ID'], keep='last')
clean_intakes.head()

In [ ]:
# Try again
clean_merge = clean_outcomes.merge(clean_intakes, on='Animal ID', how='inner')

In [ ]:
clean_merge

# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [ ]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

## See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2